Télécharger le Fichier depuis votre Machine Locale vers Colab


In [5]:
from google.colab import files

# Télécharger le fichier CSV depuis votre répertoire local
uploaded = files.upload()

# Afficher les noms des fichiers téléchargés
print(uploaded.keys())


Saving sample_submission.csv to sample_submission (1).csv
dict_keys(['sample_submission (1).csv'])


Lire le Fichier CSV dans un DataFrame pandas


In [6]:
import pandas as pd

# Extraire le nom du fichier téléchargé
filename = next(iter(uploaded))

# Charger le fichier CSV dans un DataFrame pandas
data = pd.read_csv(filename)
print(data.head())


   ImageId  Label
0        1      0
1        2      0
2        3      0
3        4      0
4        5      0


Télécharger les Fichiers train.csv et test.csv


In [7]:
# Télécharger les fichiers train.csv et test.csv
uploaded = files.upload()


Saving train.csv to train (1).csv


In [8]:
# Télécharger les fichiers test.csv
uploaded = files.upload()


Saving test.csv to test (1).csv


Lire et Préparer les Données


In [9]:
import pandas as pd
from google.colab import files


# Charger les fichiers CSV
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


Prétraitement des données

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split

# Séparer les caractéristiques et les étiquettes
X = train.drop(columns=['label'])
y = train['label']

# Normaliser les valeurs des pixels
X = X / 255.0
test = test / 255.0

# Reshape les images pour les rendre compatibles avec les réseaux de neurones convolutifs
X = X.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

# Séparer les données pour l'entraînement et la validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Pour les modèles traditionnels, aplatissez les images
X_train_flat = X_train.reshape(-1, 28*28)
X_val_flat = X_val.reshape(-1, 28*28)
test_flat = test.reshape(-1, 28*28)


Définition et entraînement des modèles

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import joblib

# Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_flat, y_train)
y_val_pred_dt = dt_model.predict(X_val_flat)
dt_accuracy = accuracy_score(y_val, y_val_pred_dt)
print(f'Decision Tree Validation Accuracy: {dt_accuracy:.4f}')

# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train_flat, y_train)
y_val_pred_rf = rf_model.predict(X_val_flat)
rf_accuracy = accuracy_score(y_val, y_val_pred_rf)
print(f'Random Forest Validation Accuracy: {rf_accuracy:.4f}')

# Support Vector Machine (SVM)
svm_model = SVC()
svm_model.fit(X_train_flat, y_train)
y_val_pred_svm = svm_model.predict(X_val_flat)
svm_accuracy = accuracy_score(y_val, y_val_pred_svm)
print(f'SVM Validation Accuracy: {svm_accuracy:.4f}')


Decision Tree Validation Accuracy: 0.8531
Random Forest Validation Accuracy: 0.9639
SVM Validation Accuracy: 0.9735


Comparer les performances des modèles


In [14]:
# Comparer les performances des modèles
model_accuracies = {
    "Decision Tree": dt_accuracy,
    "Random Forest": rf_accuracy,
    "SVM": svm_accuracy
}

best_model_name = max(model_accuracies, key=model_accuracies.get)
best_model_accuracy = model_accuracies[best_model_name]

print(f'meilleur Modele: {best_model_name} avec accuracy {best_model_accuracy:.4f}')


meilleur Modele: SVM avec accuracy 0.9735


 Sauvegarder le meilleur modèle


In [16]:
# Sauvegarder le meilleur modèle
best_model = None
if best_model_name == "Decision Tree":
    best_model = dt_model
elif best_model_name == "Random Forest":
    best_model = rf_model
elif best_model_name == "SVM":
    best_model = svm_model

joblib.dump(best_model, 'best_digit_recognition_model.pkl')
print(f"meilleur modele '{best_model_name}' enregistré comme 'best_digit_recognition_model.pkl'.")


meilleur modele 'SVM' enregistré comme 'best_digit_recognition_model.pkl'.


Faire des prédictions sur l'ensemble de test


In [17]:
# Utiliser le meilleur modèle pour faire des prédictions sur l'ensemble de test
y_test_pred = best_model.predict(test_flat)


Préparer le fichier de soumission

In [18]:
# Préparer le fichier de soumission
submission = pd.DataFrame({'ImageId': range(1, len(y_test_pred) + 1), 'Label': y_test_pred})
submission.to_csv('submission.csv', index=False)
print("Submission file created.")


Submission file created.
